# **Visualisation et statistiques descriptives**
---

## Introduction<a class="anchor" id="partie1"></a>

Ce notebook contient les codes nécessaires à la visualisation et les statistiques descriptives de notre base de données.

## Sommaire

* [Introduction](#partie1)
* [Statistiques descriptives](#partie2)
    * [Visualisation et nettoyage des données manquantes](#partie21)
    * [Statistiques descriptives intra-pays](#partie22)
        * [Résumé statistique des variables](#partie221)
        * [Étude des corrélations](#partie222)
    * [Statistiques descriptives inter-pays](#partie23)
    * [Une première analyse exploratoire des séries temporelles](#partie24)
        * [Données du PIB](#partie241)
        * [Données des Working Hours](#partie242)
        * [Données des taux d'intérêt à long terme](#partie243)
        * [Données des taux d'intérêt à court terme](#partie244)
        * [Données du CPI](#partie235)
        * [Données de la croissance potentielle](#partie246)
* [Visualisation](#partie3)
    * [Données de PIB](#partie31)
    * [Données d'heures de travail](#partie32)


In [1]:
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [openpyxl]1/2 [openpyxl]


In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
import statsmodels.api as sm
import statsmodels.tsa.api as tsa
import statsmodels.graphics.tsaplots as tsaplots
from scipy.signal import savgol_filter
from statsmodels.tsa.stattools import adfuller
import re

from Codes.scrpits_visualisations import (
    tracer_evolution,
    camembert,
    comparer_periodes
)


In [3]:
# Importation des données
df = pd.read_excel("Données_produites/base_de_données_v1.xlsx")


# Indexation
df = df.set_index('Unnamed: 0')



## Statistiques descriptives <a class="anchor" id="partie2"></a>

### Visualisation des données manquantes <a class="anchor" id="partie21"></a>

Dans une volonté de rendre compte des valeurs manquantes à notre dataframe nous créons un dataframe qui comprend l'ensemble des valeurs manquantes 

In [ ]:
df_missing_binary = df.isnull().astype(int)
df_missing_binary

In [ ]:
df_nan = df.isna()
df_nan = pd.DataFrame({
    "TIME_PERIOD_WITH_NAN": df_nan.apply(lambda col: list(df.index[col]), axis = 0)
})
df_nan.to_csv("df_nan.csv", index=True, encoding="utf-8")
print(df_nan)

In [ ]:
# Définir la figure avec un affichage interactif
fig, ax = plt.subplots(figsize=(50, 80))

# Utiliser seaborn pour une meilleure visualisation sans grille
sns.heatmap(df_missing_binary, cmap="OrRd", cbar=False, linewidths=0, ax=ax, square=False)

# Ajouter un titre
ax.set_title("Matrice des valeurs manquantes (Rouge = Manquant)")

# Rotation des labels pour une meilleure lisibilité
plt.xticks(rotation=90)
plt.yticks(rotation=0)

# Afficher la figure
plt.show()

In [ ]:
#Pourcentage de données manquantes par variable
percentage_missing=((df_missing_binary.sum())/df_missing_binary.shape[0])*100

#On ne garde que les colonnes ayant des missing values
percentage_missing=percentage_missing[percentage_missing>0]
percentage_missing = percentage_missing.sort_values(ascending=False)

'''
Nous avons un problème avec l'Estonie, 85% des données pour les taux d'intérêt sont des valeurs manquantes
J'ai revérifié sur le site de l'OCDE et c'est un problème qui vient de leur bases de données. Comme c'est une 
un pourcentage considérable, on devrait peut-être retirer l'Estonie de notre analyse

Pour les données qui sont "entrecoupées" (on dirait un chemin de la route)--> c'est les Working Hours. On peut appliquer le même 
interpolation qu'on a faite pour construire le df. 

--> Voir tentative d'interpolation dans le notebook data.ipynb
Un problème persiste avec les données qui sont au début ou à la fin de la période temporelle, il faudra envisager
une autre méthode
'''
percentage_missing


### Nettoyage des données manquantes <a class="anchor" id="partie21"></a>

Ce qui suit est une proposition

In [4]:
#Fonction qui construit un vecteur contenant le nom des variables
def variables_pays(pays):
    variable_pays=[f'CPI_{pays}',f'PIB_{pays}',f'LT_IR_{pays}',f'ST_IR_{pays}',f'WH_{pays}',f'P_Growth_{pays}']
    return(variable_pays)

In [5]:
#On élimine l'année 1995 pour éviter de faire du "backcasting" du CPI de tous les pays
df_clean=df.copy()
df_clean=df_clean.drop(["1995-Q1","1995-Q2","1995-Q3","1995-Q4"])

#On élimine l'Estonie de notre dataset
Estonia=variables_pays("Estonia")
df_clean=df_clean.drop(Estonia, axis=1)
df_clean


,CPI_Austria,PIB_Austria,LT_IR_Austria,ST_IR_Austria,WH_Austria,P_Growth_Austria,CPI_Belgium,PIB_Belgium,LT_IR_Belgium,ST_IR_Belgium,...,inflation_Norway,inflation_Poland,inflation_Portugal,inflation_Romania,inflation_Slovakia,inflation_Slovenia,inflation_Spain,inflation_Sweden,inflation_Switzerland,inflation_United
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
1996-Q1,71.553333,44641.2,6.396633,3.476667,39.8,2.573010,70.003333,54240.0,6.640000,3.350000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996-Q2,71.716667,46885.1,6.505600,3.266667,39.9,2.577419,70.516667,55590.7,6.713333,3.243333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996-Q3,71.763333,46969.6,6.430133,3.420000,40.0,2.580149,70.446667,53541.2,6.616667,3.230000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996-Q4,72.053333,47418.5,5.953500,3.330000,40.1,2.581239,70.936667,56570.4,6.000000,3.043333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997-Q1,72.463333,44369.0,5.709800,3.323333,40.2,2.580730,71.246667,53907.0,5.796667,3.203333,...,3.269231,17.188694,2.534330,116.062802,6.014689,8.149648,2.536778,1.157825,NaN,1.86001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-Q4,132.593333,122280.3,3.125533,3.957456,32.9,NaN,127.550000,158085.7,3.150000,3.957456,...,4.289056,6.285155,2.445435,7.408689,7.074750,4.968619,3.337594,3.019142,1.888900,0.00000
2024-Q1,133.326667,117154.2,2.835333,3.923615,34.0,NaN,129.700000,147812.8,2.890000,3.923615,...,4.220611,3.638082,2.463167,7.051174,3.632150,3.380074,3.235294,2.793953,1.281402,0.00000
2024-Q2,134.366667,120629.4,3.013533,3.808172,32.9,NaN,131.086667,154840.7,3.063333,3.808172,...,2.754612,2.915992,3.074050,5.784928,2.495091,2.370816,3.585946,2.074426,1.380807,0.00000


In [ ]:
df_clean_missing_binary = df_clean.isnull().astype(int)

# Définir la figure avec un affichage interactif
fig, ax = plt.subplots(figsize=(50, 80))

# Utiliser seaborn pour une meilleure visualisation sans grille
sns.heatmap(df_clean_missing_binary, cmap="OrRd", cbar=False, linewidths=0, ax=ax, square=False)

# Ajouter un titre
ax.set_title("Matrice des valeurs manquantes (Rouge = Manquant)")

# Rotation des labels pour une meilleure lisibilité
plt.xticks(rotation=90)
plt.yticks(rotation=0)

# Afficher la figure
plt.show()

In [ ]:
#Pourcentage de données manquantes par variable
percentage_missing_clean=((df_clean_missing_binary.sum())/df_clean_missing_binary.shape[0])*100

#On ne garde que les colonnes ayant des missing values
percentage_missing_clean=percentage_missing_clean[percentage_missing_clean>0]
percentage_missing_clean = percentage_missing_clean.sort_values(ascending=False)
percentage_missing_clean

Quelques lignes de code préliminaires

In [ ]:
#Création d'une copie de la base de données ayant pour index des DateTime
df_TS=df.copy()
df_TS.index = pd.PeriodIndex(df.index, freq='Q')  #conversion en PeriodIndex
df_TS.index = df_TS.index.to_timestamp() #Conversion en DateTime
print(type(df.index))

''' 
Ici je ne suis pas sûr:
- pour faire des analyses en séries temporelles, il faut que les index soient des datetime
- j'ai donc converti chaque trimestre en la première date de chaque trimestre. 
- pour des graphes, et pour transformer les sorties, on peut utiliser : df.index.strftime('%Y-Q%q')
- je ne sais pas s'il y a une meileure façon de faire

'''


In [ ]:
#On divise le df en 3 (correspondant à chaque période)
df_TS_before_2008 = df_TS.loc[df_TS.index < '2008-01-01'].copy()
df_TS_after_2008 = df_TS.loc[(df_TS.index >= '2008-01-01') & (df_TS.index<'2020-01-01')].copy()
df_TS_after_2020 = df_TS.loc[df_TS.index >= '2020-01-01'].copy()


In [ ]:
#Synthèse avant 2008
df_TS_before_2008.describe()

In [ ]:
#Synthèse après 2008
df_TS_after_2008.describe()

In [ ]:
#Synthèse après 2020
df_TS_after_2020.describe()

### Statistiques descriptives inter-pays <a class="anchor" id="partie23"></a>

#### Statistiques par types de variables <a class="anchor" id="partie231"></a>

In [ ]:
# Réglages d'affichage
pd.set_option('display.max_columns', None) 
pd.set_option('display.max_rows', None)    
pd.set_option('display.width', 1000) 

In [ ]:
group_dict = {
    # Économie en développement
    "Czechia": "developing",
    "Lithuania": "developing",
    "Latvia": "developing",
    "Poland": "developing",
    "Bulgaria": "developing",
    "Slovakia": "developing",
    "Hungary": "developing",
    "Slovenia": "developing",
    "Iceland": "developing",
    "Estonia": "developing",
    "Romania": "developing",

    # Grande économie développée
    "Germany": "large_developed",
    "United_Kingdom": "large_developed",
    "France": "large_developed",
    "Italy": "large_developed",
    "Spain": "large_developed",
    "Netherlands": "large_developed",

    # Petite économie développée
    "Austria": "small_developed",
    "Belgium": "small_developed",
    "Denmark": "small_developed",
    "Greece": "small_developed",
    "Finland": "small_developed",
    "Ireland": "small_developed",
    "Luxembourg": "small_developed",
    "Norway": "small_developed",
    "Portugal": "small_developed",
    "Sweden": "small_developed",
    "Switzerland": "small_developed"
}
pib_columns_developing = [f"PIB_{country}" for country, group in group_dict.items() if group == "developing"]
pib_columns_large_developed = [f"PIB_{country}" for country, group in group_dict.items() if group == "large_developed"]
pib_columns_small_developed = [f"PIB_{country}" for country, group in group_dict.items() if group == "small_developed"]

##### CPI <a class="anchor" id="partie2311"></a>

In [ ]:
colonnes_CPI = [colonne for colonne in df_TS.columns if colonne.startswith('CPI')]
CPI = df[colonnes_CPI]
CPI.describe()

On remarque que les moyennes sont assez similaires aux alentours comprises entre 91 et 95 pour la plupart des pays, on a cependant certains pays pour lesquels elles sont en dessous, avec notamment l'Islande et la Roumanie sous les 80 et la Suisse est la seule au delà des 100. On remarque aussi des écart-types plus élevées (au-dessus des 20) pour les pays d'Europe de l'Est et du Nord-est (et l'Islande), contre des écart-types compris entre 13 et 17 pour la plupart des autres pays, la Suisse faisant exception avec un écart-types de 2,33. Ces différences s'expliquent par des évolutions bien plus fortes depuis les années 1995 par ces pays de l'Est. les graphes ci-dessous illustrent cela.

Concernant les valeurs présentes, il y a 116 périodes, soit du premier trimestre de 1996 au dernier de 2024, sauf pour la Bulgarie (113), qui semble commencer en 1997, le Royaume-Uni (100), qui s'arrête en 2021, et la Suisse (81), qui commence en 2005.

In [ ]:
tracer_evolution(CPI, xlabel ="Période",ylabel = "Indice des Prix à la consommation")

Le graphique ci-dessus montre l'évolution de tous les pays et on peut observer la tendance haussière globale, pour expliciter le phénomènes de distinction on sépare en deux groupes nos pays et on les plot séparément 

In [ ]:
# On sépare en deux groupe selon la valeur de l'écart-type
CPI_description = CPI.describe()
std_values = CPI_description.loc['std']
CPI_groupe_1_indices = std_values[std_values < 18].index  
CPI_groupe_2_indices = std_values[std_values > 18].index
CPI_groupe_1 = CPI[CPI_groupe_1_indices]
CPI_groupe_2 = CPI[CPI_groupe_2_indices]

# Ajout de la valeur moyenne
Moyenne_CPI_1 = CPI_groupe_1.mean(axis=1)
CPI_groupe_1['CPI_Mean'] = Moyenne_CPI_1
Moyenne_CPI_2 = CPI_groupe_2.mean(axis=1)
CPI_groupe_2['CPI_Mean'] = Moyenne_CPI_2

CPI_Moyenne = pd.DataFrame()
CPI_Moyenne["CPI_Mean_group_1"] = Moyenne_CPI_1
CPI_Moyenne["CPI_Mean_group_2"] = Moyenne_CPI_2

In [ ]:
tracer_evolution(CPI_groupe_1, xlabel ="Période",ylabel = "Indice des Prix à la consommation des pays avec une plus faible évolution")

In [ ]:
tracer_evolution(CPI_groupe_2, xlabel ="Période",ylabel = "Indice des Prix à la consommation (Std < 18)")

In [ ]:
tracer_evolution(CPI_Moyenne, xlabel ="Période",ylabel = "Indice des Prix à la consommation moyen entre pour les deux groupes")

On observe mieux les différences entre le groupe 1 et le groupe 2, avec une croissance de CPI moyenne plus élevée dans le second groupe. On va essayer de voir si la différence entre ces deux groupes est aussi marquée pour les autres variables.

On va s'intéresser aux statistiques en glissement annuel. 

In [ ]:
CPI_Glissement = CPI.pct_change(periods=4) * 100 
CPI_Glissement.describe()

L’analyse des variations annuelles du CPI (approximation de l’inflation) pour les pays européens met en évidence des disparités marquées entre les différentes économies. Les moyennes d’inflation révèlent une nette distinction entre les pays d’Europe de l’Ouest et du Nord, à inflation modérée et relativement stable, et les pays d’Europe centrale et orientale, caractérisés par une inflation plus élevée. Par exemple, la Roumanie présente une moyenne exceptionnelle de 17,4 %, suivie de la Bulgarie (9,9 %), la Hongrie (6,2 %) ou encore les États baltes comme l’Estonie (4,5 %) et la Lettonie (4,2 %). À l’opposé, des pays comme la Suisse (0,52 %), le Royaume-Uni (1,66 %), la France (1,80 %) ou encore la Finlande (1,86 %) affichent des taux d’inflation moyens nettement inférieurs, illustrant une gestion plus stable des prix à la consommation.
Cette hétérogénéité se retrouve également dans la volatilité des séries. Les écarts-types confirment la forte instabilité de l’inflation dans certaines économies émergentes, notamment en Roumanie (écart-type de 31,2) et en Bulgarie (52,2), où l’on observe aussi les valeurs extrêmes les plus aberrantes (maximum à 544 % en Bulgarie, par exemple). À l’inverse, des pays comme la Suisse, l’Irlande ou la France montrent une inflation bien plus contenue et prévisible. Le minimum de certaines séries est même négatif, ce qui reflète des épisodes de désinflation ponctuelle dans plusieurs pays (ex : Irlande, Grèce, Islande).
Les indicateurs de distribution (quartiles) permettent d’observer la dispersion typique des taux d’inflation. Les pays développés ont une distribution resserrée autour d’une médiane proche de 2 %, avec des 25e et 75e percentiles proches (ex : Autriche, Belgique, France). En revanche, les pays émergents présentent des distributions très étendues, témoignant d’une plus grande exposition aux chocs de prix.
En somme, cette analyse confirme l’existence de trois régimes d’inflation en Europe : un régime stable à faible inflation (économies développées d’Europe de l’Ouest et du Nord), un régime modéré mais plus volatil (certains pays d’Europe du Sud ou périphériques), et un régime d’inflation forte et instable (économies émergentes d’Europe de l’Est). 

##### PIB <a class="anchor" id="partie2312"></a>

In [ ]:
colonnes_PIB = [colonne for colonne in df_TS.columns if colonne.startswith('PIB')]
PIB = df[colonnes_PIB]
PIB.describe()

Concernant les valeurs présentes  il y en a entre 118 et 120 pour tous les pays, c'est à dire toutes les périodes sauf une ou deux, et le Royaume-Uni n'en a que 103, qui s'arrête en 2020, comme pour le CPI.

Ici les moyennes diffèrent grandement tout comme les écart-types. On va donc regarder l'évolution des PIB. 

In [ ]:
tracer_evolution(PIB, xlabel ="Période",ylabel = "PIB en Millions ")

On voit trois choses sur ce graphiques, d'abord qu'un grand nombre de valeurs sont écrasées par les plus grosses et donc sont difficiles à observer lorsque nous sommes en échelle linéaire, ensuite les séries semblent croissantes et avec des saisons, enfin on distingue bien la période de récession du Covid.

In [ ]:
camembert(PIB, période = "2021-Q1", titre = "PIB en Millions au Q1 de 2021")

Ce diagramme fait au premier trimestre de 2020 (dernière date où le PIB du Royaume-Uni est renseignée) illustre parfaitement le fait que les plus gros pays écrase les valeurs des plus petits, en prenant l'Allemagne, le Royaume-Uni, la France, l'Italie et l'Espagne on observe déjà 70% du PIB total, de plus 11 pays comptent pour moins de 1% du total. 

In [ ]:
PIB = PIB.rename(columns=lambda x: f"LOG_{x}")
tracer_evolution(PIB, xlabel ="Période",ylabel = "log(PIB) en Millions ", log = True)

En valeurs logarithmiques on voit bien la tendance linéaire assez similaire entre les séries, tout comme les variations saisonnales. Maintenant on va séparer en deux groupes les PIB afin de pouvoir mieux observer l'effet de la récession du covid. 

In [ ]:
# On sépare en deux groupe selon la valeur de la moyenne
PIB_description = PIB.describe()
PIB_mean_values = PIB_description.loc['mean']
PIB_groupe_1_indices = PIB_mean_values[PIB_mean_values > 150000].index  
PIB_groupe_2_indices = PIB_mean_values[PIB_mean_values < 150000].index
PIB_groupe_1 = PIB[PIB_groupe_1_indices]
PIB_groupe_2 = PIB[PIB_groupe_2_indices]

In [ ]:
tracer_evolution(PIB_groupe_1, xlabel ="Période",  start_date="2019-Q2", end_date="2021-Q2", ylabel = "PIB en Millions ")

In [ ]:
tracer_evolution(PIB_groupe_2, xlabel ="Période",  start_date="2019-Q2", end_date="2021-Q2", ylabel = "PIB en Millions ")

En séparant en deux groupes on constate moins la diminution due au Covid au Q2 de 2020 parmi le groupe dont le PIB moyen est en dessous de 200 Milliards.

In [ ]:
comparer_periodes(PIB, "2020-Q1", "2020-Q2")

In [ ]:
print(comparer_periodes(PIB, "2020-Q1", "2020-Q2")['Différence de %'].mean())

En observant la différence entre les deux premiers trimestres de 2020 on observe très bien la récession du confinement. La récession moyenne est de -5.12%.

Pour revenir à la séparation des groupes faites lors de l'analyse du CPI, on remarque que tous les pays du groupes 2 précédent sont encore dans le groupe 2 en terme de PIB et que certains pays du groupe 1 sont maintenant dans ce groupe, ce sont surtout les pays d'Europe Centrale et du Nord ayant des populations plus faibles. On pourrait donc être amené à distinguer par la suite trois groupes, les économies qui en train de se développer, les petites économies développées et les grandes économies développées. 

In [ ]:
print(PIB_groupe_2_indices, CPI_groupe_2_indices)

On va s'intéresser aux statistiques en glissement annuel. 

In [ ]:
PIB_Glissement = PIB.pct_change(periods=4) * 100 
PIB_Glissement.describe()

L’étude des variations annuelles du PIB met en lumière des différences nettes entre les pays européens, reflétant à la fois le niveau de développement économique et la sensibilité aux chocs conjoncturels. En moyenne, les pays d’Europe de l’Est et du Nord, souvent classés comme économies émergentes ou plus dynamiques, affichent des taux de croissance bien supérieurs. On retrouve ainsi la Lituanie (10,3 %), la Lettonie (8,8 %), l’Irlande (8,6 %), l’Estonie (9,6 %) ou encore la Bulgarie (8,6 %) avec des niveaux de croissance annualisée élevés. Ces performances moyennes témoignent soit de rattrapages économiques structurels, soit d’une plus grande exposition aux cycles d’expansion.
À l’inverse, les grandes économies développées comme l’Allemagne (2,75 %), la France (3,1 %), l’Italie (3,2 %) ou le Royaume-Uni (3,2 %) présentent des taux de croissance plus modérés, mais souvent plus réguliers, confirmant une certaine stabilité structurelle. Cela est cohérent avec le rôle de ces économies en tant que piliers de la zone euro, mais aussi avec leur maturité économique et leur moindre potentiel de rattrapage.
La dispersion des données confirme ces dynamiques : les écarts-types les plus élevés se retrouvent dans les pays à forte croissance moyenne, comme la Roumanie, la Hongrie, la Slovaquie ou les États baltes, qui sont également ceux ayant connu des chocs sévères ou des reprises rapides. Des valeurs extrêmes, positives ou négatives, sont observées dans ces pays. Par exemple, l’Irlande connaît une croissance maximale de 38,8 %, et la Bulgarie de 136 %, ce qui indique soit des anomalies statistiques ponctuelles (souvent liées à des bases faibles), soit des fluctuations extrêmes dues à des événements exogènes. À l’opposé, les pays comme la Suisse, l’Autriche ou la Belgique présentent des profils de croissance plus encadrés, avec une moindre amplitude entre les quartiles.
Les indicateurs de position (médiane, quartiles) renforcent cette lecture : les économies développées ont des médianes de croissance comprises entre 2,9 % et 3,5 %, tandis que les économies en développement ou de petite taille montrent des médianes plus élevées (jusqu’à 10 %), mais aussi plus dispersées. Enfin, certains pays comme la Norvège, le Luxembourg ou l’Irlande présentent des caractéristiques mixtes : haut niveau de développement, mais volatilité importante, probablement liée à des effets sectoriels ou à leur structure économique (ressources naturelles, multinationales, etc.).

##### Taux d'intérêts <a class="anchor" id="partie2313"></a>

In [ ]:
colonnes_LT = [colonne for colonne in df_TS.columns if colonne.startswith('LT')]
LT = df[colonnes_LT]
LT.describe()

In [ ]:
colonnes_ST = [colonne for colonne in df_TS.columns if colonne.startswith('ST')]
ST = df[colonnes_ST]
ST.describe()

Les valeurs renseignées pour les taux d'intérêts de long-termes sont assez hétérogènes. 16 pays ont toutes les valeurs de renseignées, les autres pays ont entre 91 et 110 valeurs renseignée sauf la Roumanie avec 79, la Bulgarie avec 88 et l'Estonie avec seulement 18. Concernant les taux à court-terme on a cette fois 19 pays avec toutes les valeurs renseignées et pour les autres on a seulement la Slovénie (avec 92 valeurs) et la Bulgarie (avec 83 valeurs) sous les 100 valeurs renseignée. 

Les moyennes des taux de long terme oscille entre 1,78% pour la Suisse et 7,20% pour l'Islande, avec une majorité des pays au alentour des 3-4%. (21 pays), il y a l'Estonie à 2,1 % et le reste au alentour des 5-6%. Les écart-types sont relativement proches des 2 sauf pour la Grèce avec 4,5, la Lettonie avec 3,9 et la Lituanie avec 3,6.

Pour les taux de court terme les moyennes sont aux environs des 2% pour la plupart des pays des pays d'Europe de l'Ouest et du Nord, et sont plus hautes en moyenne pour les pays d'Europe de l'Est avec notamment la Roumanie et ses 21% de moyenne, la Hongrie avec ses 8,8% et la Pologne avec ses 7,8%. les écart-types sont relativement proches des valeurs moyennes.

In [ ]:
tracer_evolution(LT, xlabel ="Période", ylabel = "Taux de long terme")

In [ ]:
tracer_evolution(ST, xlabel ="Période", ylabel = "Taux de court terme")

In [ ]:
ST_wo_Roumania = ST.drop(columns = ["ST_IR_Romania"])
tracer_evolution(ST_wo_Roumania, xlabel ="Période", ylabel = "Taux de court terme")

On sort la Roumanie de la représentation des taux de court-terme car ses valeurs sont trop grandes. Pour les taux de long-terme on observe une corrélation entre les taux assez forte avec une tendance décroissante marquée par deux bosses en 2008 et 2012 qui correspondent à la crise des subprimes et de l'euro, puis une troisième en 2022 qui est celle qui nous intéresse pour cette étude. Concernant les taux de court-terme on a des fortes irrégularités jusqu'en 2003, puis une période de stabilisation avant une hausse en 2008 et une autre périodes stable jusqu'à la hausse de 2022.

In [ ]:
tracer_evolution(LT, xlabel ="Période", start_date="2020-Q3", end_date="2024-Q4",ylabel = "Taux de long terme")

In [ ]:
tracer_evolution(ST, xlabel ="Période", start_date="2020-Q3", end_date="2024-Q4",ylabel = "Taux de court terme")

En regardant plus précisément les périodes qui nous intéressent, on remarque que la hausse débute dès le Q1 de 2021 et semble se stabiliser à partir du Q3 de 2023.

In [ ]:
comparer_periodes(LT, "2021-Q1", "2023-Q3", percent = False)

In [ ]:
print(comparer_periodes(LT, "2021-Q1", "2023-Q3", percent = False).mean(), comparer_periodes(ST, "2021-Q1", "2023-Q3", percent = False).mean())

Ainsi on voit qu'il y a eu une hausse moyenne des taux de 3,38 points de pourcentages pour les taux de long terme et de 4,75 points de pourcentages pour les taux de court terme.

In [ ]:
LT_wo_Switzerland = LT.drop(columns = ["LT_IR_Switzerland"])
tracer_evolution(LT_wo_Switzerland, xlabel ="Période", start_date="2015-Q1", end_date="2024-Q3",ylabel = "Variation du taux de long terme par rapport à la base du Q1 de 2015", base = "2015-Q1", lissage = True)

On observe bien sur ce graphique en avec comme base le premier trimestre de 2015 la hausse soudaine des taux dès le premier trimestre de 2021 et ce jusqu'au deuxième, voire troisième trimestre de 2023. Et ce pour tous les pays, on ne le voit pas pour la Grèce sur ce graphique, mais cela est dû à sa base qui est très élevé en 2015 étant en train de sortir d'une crise de la dette, dont on a observé les hausses massives de taux sur les graphiques précédents.

##### Heures travaillées <a class="anchor" id="partie2314"></a>

In [ ]:
colonnes_WH = [colonne for colonne in df_TS.columns if colonne.startswith('WH')]
WH = df[colonnes_WH]
WH.describe()

Pour les heures travaillées tous les pays ont au moins 100 valeurs renseignées avec 11 pays sur 28 qui n'ont pas toute leurs valeurs. Les moyennes oscillent entre 34 et 40, sauf pour les Pays-Bas avec 31 heures et il y a 4 pays au dessus des 40 heures, à savoir la Bulgarie, la Tchéquie, la Grèce et la Pologne. Les écarts-types sont aussi faibles.

In [ ]:
tracer_evolution(WH, xlabel ="Période", ylabel = "Évolution des heures/semaine travaillées")

Les données sont vraiment exploitables à partir de 1999 et semble suivre une tendance décroissante et être très impactées par la saisonnalité, ce qu'il faudra vérifier. 

##### Croissance potentielle <a class="anchor" id="partie2315"></a>

In [ ]:
colonnes_PG = [colonne for colonne in df_TS.columns if colonne.startswith('P_')]
PG = df[colonnes_PG]
PG.describe()

Les données ont été obtenue par interpolation polynomiale donc on a toutes les valeurs jusqu'à 2021 là où s'arrêtais les valeurs de la série initiale. Les moyennes sont comprises entre 1,2 et 4 avec l'Ireland à 5 et l'Italie à 0.5.

In [ ]:
tracer_evolution(PG, xlabel ="Période", ylabel = "Évolution de la croissance potentielle")

On voit très bien l'interpolation polynomiale des variables et on perçoit une phase de baisse de la croissance potentielle entre 2003 et 2016 avant de remonter plus nettement.

#### Étude des corrélations <a class="anchor" id="partie222"></a>

Correlations\
Idée: nous allons étudier les correlations entre plusieurs variables de 4 pays (corrélations à l'intérieur du même pays)

In [ ]:
France=variables_pays('France')
Germany=variables_pays('Germany')
Denmark=variables_pays('Denmark')
Romania=variables_pays('Romania')

plt.figure(figsize=(20, 20))

plt.subplot(2,2,1)
sns.heatmap(df[France].corr())
plt.title("Matrice de corrélation des variables de la France")

plt.subplot(2,2,2)
sns.heatmap(df[Germany].corr())
plt.title("Matrice de corrélation des variables de l'Allemagne")

plt.subplot(2,2,3)
sns.heatmap(df[Denmark].corr())
plt.title("Matrice de corrélation des variables du Danemark")

plt.subplot(2,2,4)
sns.heatmap(df[Romania].corr())
plt.title("Matrice de corrélation des variables de la Roumanie")



#### Étude comparatives des variables <a class="anchor" id="partie222"></a>

On va faire la moyenne par groupe des différentes variables puis les comparer

In [ ]:
# On passe le PIB en glissement annuel
pib_cols = [col for col in df.columns if col.startswith('PIB')]
df[pib_cols] = df[pib_cols].pct_change(periods=4) * 100


In [ ]:
cols_to_keep = [col for col in df.columns if not col.startswith('CPI_')]
groups = ["developing", "small_developed", "large_developed"]
variables = ["inflation", "PIB", "ST_IR", "LT_IR", "P_Growth"]
df_grouped = pd.DataFrame(index=df.index)

# Création d'un dataframe avec tous les groupes et toutes les variables
for var in variables:
    for group_name in groups:
        relevant_cols = []
        for country, grp in group_dict.items():
            if grp == group_name:
                col_name = f"{var}_{country}"
                if col_name in df.columns:
                    relevant_cols.append(col_name)
        if relevant_cols:
            df_grouped[f"{var}_{group_name}"] = df[relevant_cols].mean(axis=1)

# Création d'un df par groupe
for group in groups:
    cols = [col for col in df_grouped.columns if col.endswith(f"_{group}")]
    globals()[f"df_{group}"] = df_grouped[cols].copy()

In [ ]:
df_developing.describe()

In [ ]:
df_small_developed.describe()

In [ ]:
df_large_developed.describe()

In [ ]:
correlation_matrix = df_grouped.corr()
print(correlation_matrix)

In [ ]:
plt.figure(figsize=(12, 10))
sns.heatmap(df_grouped.corr(), annot=True, cmap="coolwarm", center=0)
plt.title("Matrice des corrélations entre variations économiques par groupe")
plt.tight_layout()
plt.show()

In [ ]:
# Construction d'un nouveau DataFrame pour les boxplots
df_boxplot = df_grouped[[
    "inflation_developing", "inflation_small_developed", "inflation_large_developed",
    "PIB_developing", "PIB_small_developed", "PIB_large_developed"
]]

# Melt pour seaborn
df_boxplot_melted = df_boxplot.melt(var_name='Variable_Groupe', value_name='Variation (%)')

plt.figure(figsize=(14, 6))
sns.boxplot(x="Variable_Groupe", y="Variation (%)", data=df_boxplot_melted)
plt.xticks(rotation=45)
plt.title("Dispersion des variations annuelles d'inflation et de PIB par groupe")
plt.tight_layout()
plt.show()

In [ ]:
# Reprise du DataFrame initial
df_scatter = df_grouped[[
    "inflation_developing", "PIB_developing",
    "inflation_small_developed", "PIB_small_developed",
    "inflation_large_developed", "PIB_large_developed"
]].dropna()

# Définir un seuil raisonnable (par ex. 99e percentile) pour éliminer les points extrêmes
def remove_outliers(df, x_col, y_col, x_thresh=0.99, y_thresh=0.99):
    x_limit = df[x_col].quantile(x_thresh)
    y_limit = df[y_col].quantile(y_thresh)
    x_min = df[x_col].quantile(1 - x_thresh)
    y_min = df[y_col].quantile(1 - y_thresh)
    return df[(df[x_col] < x_limit) & (df[x_col] > x_min) &
              (df[y_col] < y_limit) & (df[y_col] > y_min)]

# Appliquer le filtre par groupe
df_d = remove_outliers(df_scatter, "inflation_developing", "PIB_developing")
df_s = remove_outliers(df_scatter, "inflation_small_developed", "PIB_small_developed")
df_l = remove_outliers(df_scatter, "inflation_large_developed", "PIB_large_developed")

# Tracé
plt.figure(figsize=(10, 7))
plt.scatter(df_d["inflation_developing"], df_d["PIB_developing"], label="Developing", alpha=0.7, color="red")
plt.scatter(df_s["inflation_small_developed"], df_s["PIB_small_developed"], label="Small Developed", alpha=0.7, color="blue")
plt.scatter(df_l["inflation_large_developed"], df_l["PIB_large_developed"], label="Large Developed", alpha=0.7, color="green")

plt.axhline(0, color='grey', linestyle='--')
plt.axvline(0, color='grey', linestyle='--')
plt.xlabel("Variation annuelle de l'inflation (%)")
plt.ylabel("Variation annuelle du PIB (%)")
plt.title("Lien entre variations annuelles de l'inflation et du PIB (hors valeurs extrêmes)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
# Sélection des colonnes à tracer
columns_to_plot = [
    "inflation_developing", "PIB_developing",
    "inflation_small_developed", "PIB_small_developed",
    "inflation_large_developed", "PIB_large_developed"
]

# Redéfinition manuelle de la palette personnalisée pour garantir la cohérence groupe-variable
custom_colors = {
    "inflation_developing": "red",
    "PIB_developing": "darkred",
    "inflation_small_developed": "blue",
    "PIB_small_developed": "navy",
    "inflation_large_developed": "green",
    "PIB_large_developed": "darkgreen"
}

# Injection temporaire de la palette dans la fonction via monkey patching simple
def tracer_evolution_custom(df, columns=None, xlabel=None, ylabel=None, start_date=None, end_date=None, log=False, base=None, lissage=False):
    import numpy as np
    import matplotlib.pyplot as plt
    from scipy.signal import savgol_filter
    import matplotlib.ticker as ticker

    if start_date and end_date:
        date = (df.index >= start_date) & (df.index <= end_date)
        df_filtered = df.loc[date]
    elif start_date:
        date = (df.index >= start_date)
        df_filtered = df.loc[date]
    elif end_date:
        date = (df.index <= end_date)
        df_filtered = df.loc[date]
    else:
        df_filtered = df

    if columns:
        df_filtered = df_filtered[columns]
    else:
        columns = df_filtered.columns

    mean_values = df_filtered.mean().sort_values(ascending=False)
    sorted_columns = mean_values.index
    df_filtered = df_filtered[sorted_columns]

    if log:
        df_filtered = df_filtered.applymap(lambda x: np.log(x) if x > 0 else np.nan)
    
    if base:
        df_filtered = df_filtered / df.loc[base] - 1

    if lissage:
        for col in df_filtered.columns:
            df_filtered[col] = savgol_filter(df_filtered[col], window_length=10, polyorder=3)

    plt.figure(figsize=(14, 6))

    for col in df_filtered.columns:
        plt.plot(df_filtered.index, df_filtered[col],
                 label=col,
                 color=custom_colors.get(col, None))

    if xlabel:
        plt.xlabel(xlabel)
    if ylabel:
        plt.ylabel(ylabel)

    num_ticks = 8
    indices = np.linspace(0, len(df_filtered.index) - 1, num_ticks, dtype=int)
    plt.xticks(df_filtered.index[indices], rotation=45)

    plt.legend(loc="upper left", bbox_to_anchor=(1, 1), ncol=2)
    plt.grid(True)
    plt.gca().yaxis.set_major_formatter(ticker.StrMethodFormatter("{x:,.0f}"))
    plt.tight_layout()
    plt.title("Variations annuelles de l'inflation et du PIB par groupe")
    plt.show()

# Appel de la fonction avec df_grouped
tracer_evolution_custom(
    df_grouped,
    columns=columns_to_plot,
    xlabel="Date",
    ylabel="Variation annuelle (%)", 
    start_date = "2018-Q1"
)


### Une première analyse exploratoire des séries temporelles <a class="anchor" id="partie24"></a>

On fait ici une analyse automatique de la stationnarité en utilisant l'ADF (Augmented Dickey-Fuller Test)

In [24]:
df_filtered = df.loc[:, ~df.columns.str.contains("CPI")]  # Supprimer CPI

# PIB en glissement annuel 
pib_cols = [col for col in df_filtered.columns if col.startswith('PIB')]
df_filtered[pib_cols] = df_filtered[pib_cols].pct_change(periods=4) * 100
df_filtered.rename(columns={'inflation_United': 'inflation_United_Kingdom'}, inplace=True)



variables = ['PIB', 'inflation', 'ST_IR', 'LT_IR', 'WH', 'P_Growth']
selected_cols = [col for col in df_filtered.columns if any(col.startswith(var) for var in variables)]
pays = set()
for col in selected_cols:
    for var in variables:
        pattern = f"^{var}_(.+)$"
        match = re.match(pattern, col)
        if match:
            pays.add(match.group(1))
            break

pays = sorted(pays)

# --- 5. Initialiser le résultat stationnarité ---
stationarity_df = pd.DataFrame(index=pays, columns=variables)

# --- 6. Tester la stationnarité ADF pour chaque variable par pays ---
for country in stationarity_df.index:
    for var in variables:
        col_name = f"{var}_{country}"
        if col_name in df_filtered.columns:
            serie = df_filtered[col_name].dropna()
            if len(serie) > 20:  # nombre minimal d'observations
                result = adfuller(serie, regression='c', autolag='AIC')
                pval = result[1]
                stationarity_df.loc[country, var] = int(pval < 0.1)
            else:
                stationarity_df.loc[country, var] = '?'
        else:
            stationarity_df.loc[country, var] = '?'

/tmp/ipykernel_3350/2946139260.py:5: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df_filtered[pib_cols] = df_filtered[pib_cols].pct_change(periods=4) * 100
/tmp/ipykernel_3350/2946139260.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered[pib_cols] = df_filtered[pib_cols].pct_change(periods=4) * 100
/tmp/ipykernel_3350/2946139260.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ret

In [25]:
stationarity_df

,PIB,inflation,ST_IR,LT_IR,WH,P_Growth
Austria,1,0,0,0,0,0
Belgium,1,0,1,0,0,0
Bulgaria,1,1,0,0,0,0
Czechia,1,1,0,1,0,0
Denmark,1,0,1,0,0,0
Estonia,1,0,1,?,0,0
Finland,1,1,1,0,0,0
France,1,1,1,0,1,0
Germany,1,0,0,0,0,0
Greece,0,0,0,1,0,0


Pour chaque variable, nous allons déterminer  le tendances et les saisonalités des séries temporelles. Compte tenu du nombre de pays dans notre échantillon, nous allons tout d'abord faire cette analyse pour la France et pour les principaux pays d'Europe. 

#### Données du PIB <a class="anchor" id="partie241"></a>

In [ ]:
#Fonction qui effectue la représentation de l'évolution du PIB d'un seul pays. 

def graphe_PIB(pays): 
    plt.figure(figsize=(14, 6))  # Taille du graphique

    plt.plot(df_TS.index, df_TS[f'PIB_{pays}'], marker='o', linestyle='-')

    plt.xlabel("Période")
    plt.ylabel("PIB en millions")
    plt.title(f"Évolution du PIB - {pays} (1995-2024)")

    #   Sélectionner un sous-ensemble des labels de l'axe X pour éviter la surcharge
    num_ticks = 8  # Nombre de labels affichés sur l'axe X
    indices = np.linspace(0, len(df_TS.index) - 1, num_ticks, dtype=int)  # Espacement des indices
    plt.xticks(df_TS.index[indices],df_TS.index[indices].to_period('Q').strftime('%Y-Q%q'),rotation=45)  # Appliquer les labels espacés

    plt.grid(True)
    return(plt.show())


In [ ]:
#Représentation graphique de plusieurs pays
selected_countries = ['Austria', 'France', 'United_Kingdom','Spain', 'Italy', 'Netherlands', 'Switzerland'] #Ici pon peut choisir les pays que l'on veut
df_selected = df.loc[:,[f'PIB_{pays}' for pays in selected_countries]]

colors = plt.cm.get_cmap("tab10", len(df_selected.columns))  # Utilisation d'une palette de couleurs
plt.figure(figsize=(14, 6))

for i, country in enumerate(df_selected.columns): #enumerate permet de sélectionner l'élément et son indice
    plt.plot(df_selected.index, df_selected[country], label=country, color=colors(i))

# Personnalisation du graphique
plt.xlabel("Période")
plt.ylabel("PIB en Millions")
plt.title("Principaux PIB trimestriels en Europe ")

# Indicage
num_ticks = 8 
indices = np.linspace(0, len(df_selected.index) - 1, num_ticks, dtype=int)  
plt.xticks(df_selected.index[indices], rotation=45) 

# Affichage 
plt.legend(loc="upper left", bbox_to_anchor=(1, 1), ncol=2)  # Légende en dehors du graphique
plt.grid(True)
plt.gca().yaxis.set_major_formatter(ticker.StrMethodFormatter("{x:,.0f}")) #permet de ne pas avoir d'écriture scientifique
plt.show()

In [ ]:
#Représentation de la saisonalité
plt.figure()
tsaplots.quarter_plot(df_TS['PIB_France'].resample('Q').mean())
plt.show()

A première vue, il semblerait qu'il n'y ait pas de saisonalité pour le PIB

Ce qui suit est un brouillon qui nous sera utile lorsque nous aurons déterminé la stationnarité de la série temporelle

In [ ]:
#Autocorrélation (ACF)
plt.figure(figsize=(14,6))
tsaplots.plot_acf(df_TS['PIB_France'], lags=30)
plt.title("Fonction d'autocorrélation du PIB de la France")
plt.show()

#Remarque: on n'a pas encore vérifié que la série est stationnaire, donc l'intévalle de confiance peut-être biaisé


In [ ]:
#Autocorrélation partielle (PACF)
plt.figure(figsize=(14,6))
tsaplots.plot_pacf(df_TS['PIB_France'], lags=30)
plt.title("Fonction d'autocorrélation partielle du PIB de la France")
plt.show()

#### Données des Working Hours <a class="anchor" id="partie242"></a>

In [ ]:
#Représentation graphique
plt.figure(figsize=(14, 6))  # Taille du graphique

plt.plot(df_TS.index, df_TS['WH_France'], marker='o', linestyle='-')

plt.xlabel("Période")
plt.ylabel("Nombre d'heures travaillées par semaine")
plt.title("Évolution du nombre d'heures travaillées en France(1995-2024)")

# Sélectionner un sous-ensemble des labels de l'axe X pour éviter la surcharge
num_ticks = 8  # Nombre de labels affichés sur l'axe X
indices = np.linspace(0, len(df_TS.index) - 1, num_ticks, dtype=int)  # Espacement des indices
plt.xticks(df_TS.index[indices],df_TS.index[indices].to_period('Q').strftime('%Y-Q%q'),rotation=45)  # Appliquer les labels espacés

plt.grid(True)
plt.show()

La tendance décroissance du nombre d'heures travaillées en France peut s'expliquer par
- l'interpolation faite entre 1995-2002 (données annuelles disponibles uniquement)
- la mise en place des lois (nombre d'heures qui passe de 39 à 35)

#### Données des taux d'intérêt à long terme <a class="anchor" id="partie243"></a>

In [ ]:
#Représentation graphique
plt.figure(figsize=(14, 6))  # Taille du graphique

plt.plot(df_TS.index, df_TS['LT_IR_France'], marker='o', linestyle='-')

plt.xlabel("Période")
plt.ylabel("Taux d'intérêt (Long Terme) de la France")
plt.title("Évolution du taux d'intérêt (long terme) de la France (1995-2024)")

# Sélectionner un sous-ensemble des labels de l'axe X pour éviter la surcharge
num_ticks = 8  # Nombre de labels affichés sur l'axe X
indices = np.linspace(0, len(df_TS.index) - 1, num_ticks, dtype=int)  # Espacement des indices
plt.xticks(df_TS.index[indices],df_TS.index[indices].to_period('Q').strftime('%Y-Q%q'),rotation=45)  # Appliquer les labels espacés

plt.grid(True)
plt.show()

In [ ]:
#Représentation de la saisonalité
plt.figure()
tsaplots.quarter_plot(df_TS['LT_IR_France'].resample('Q').mean())
plt.show()

#### Données des taux d'intérêt à court terme <a class="anchor" id="partie244"></a>

In [ ]:
#Représentation graphique
plt.figure(figsize=(14, 6))  # Taille du graphique

plt.plot(df_TS.index, df_TS['ST_IR_France'], marker='o', linestyle='-')

plt.xlabel("Période")
plt.ylabel("Taux d'intérêt (court terme)")
plt.title("Évolution du taux d'intérêt (court terme) de la France (1995-2024)")

# Sélectionner un sous-ensemble des labels de l'axe X pour éviter la surcharge
num_ticks = 8  # Nombre de labels affichés sur l'axe X
indices = np.linspace(0, len(df_TS.index) - 1, num_ticks, dtype=int)  # Espacement des indices
plt.xticks(df_TS.index[indices],df_TS.index[indices].to_period('Q').strftime('%Y-Q%q'),rotation=45)  # Appliquer les labels espacés

plt.grid(True)
plt.show()

In [ ]:
#Représentation de la saisonalité
plt.figure()
tsaplots.quarter_plot(df_TS['ST_IR_France'].resample('Q').mean())
plt.show()

#### Données du CPI <a class="anchor" id="partie245"></a>

In [ ]:
#Représentation graphique
plt.figure(figsize=(14, 6))  # Taille du graphique

plt.plot(df_TS.index, df_TS['PIB_France'], marker='o', linestyle='-')

plt.xlabel("Période")
plt.ylabel("CPI")
plt.title("Évolution du du CPI de la France (1995-2024)")

# Sélectionner un sous-ensemble des labels de l'axe X pour éviter la surcharge
num_ticks = 8  # Nombre de labels affichés sur l'axe X
indices = np.linspace(0, len(df_TS.index) - 1, num_ticks, dtype=int)  # Espacement des indices
plt.xticks(df_TS.index[indices],df_TS.index[indices].to_period('Q').strftime('%Y-Q%q'),rotation=45)  # Appliquer les labels espacés

plt.grid(True)
plt.show()

In [ ]:
#Représentation de la saisonalité
plt.figure()
tsaplots.quarter_plot(df_TS['PIB_France'].resample('Q').mean())
plt.show()

#### Données de la croissance potentielle <a class="anchor" id="partie246"></a>

In [ ]:
#Représentation graphique
plt.figure(figsize=(14, 6))  # Taille du graphique

plt.plot(df_TS.index, df_TS['P_Growth_France'], marker='o', linestyle='-')

plt.xlabel("Période")
plt.ylabel("Croissance potentielle")
plt.title("Évolution de la croissance potentielle de la France (1995-2021)")

# Sélectionner un sous-ensemble des labels de l'axe X pour éviter la surcharge
num_ticks = 8  # Nombre de labels affichés sur l'axe X
indices = np.linspace(0, len(df_TS.index) - 1, num_ticks, dtype=int)  # Espacement des indices
plt.xticks(df_TS.index[indices],df_TS.index[indices].to_period('Q').strftime('%Y-Q%q'),rotation=45)  # Appliquer les labels espacés

plt.grid(True)
plt.show()

## Visualisation <a class="anchor" id="partie3"></a>

### Données de PIB <a class="anchor" id="partie31"></a>

Pour notre rendu de mi-parcours nous avons présenté les évolutions des PIB en France et dans les principaux pays d'Europe. Nous faisons d'abord celui de la France.

In [ ]:
plt.figure(figsize=(14, 6))  # Taille du graphique

plt.plot(df.index, df['PIB_France'], marker='o', linestyle='-')

plt.xlabel("Période")
plt.ylabel("PIB de la France")
plt.title("Évolution du PIB de la France (1995-2024)")

# Sélectionner un sous-ensemble des labels de l'axe X pour éviter la surcharge
num_ticks = 8  # Nombre de labels affichés sur l'axe X
indices = np.linspace(0, len(df.index) - 1, num_ticks, dtype=int)  # Espacement des indices
plt.xticks(df.index[indices],rotation=45)  # Appliquer les labels espacés

plt.grid(True)
plt.show()

Et ensuite pour l'Europe

In [ ]:
selected_countries = ['PIB_Germany', 'PIB_France', 'PIB_United_Kingdom','PIB_Spain', 'PIB_Italy', 'PIB_Netherlands', 'PIB_Switzerland']

df_selected = df[selected_countries]

colors = plt.cm.get_cmap("tab10", len(df_selected.columns))  # Utilisation d'une palette de couleurs
plt.figure(figsize=(14, 6))

for i, country in enumerate(df_selected.columns): #enumerate permet de sélectionner l'élément et son indice
    plt.plot(df_selected.index, df_selected[country], label=country, color=colors(i))

# Personnalisation du graphique
plt.xlabel("Période")
plt.ylabel("PIB en Millions")
plt.title("Principaux PIB trimestriels en Europe ")

# Indicage
num_ticks = 8 
indices = np.linspace(0, len(df_selected.index) - 1, num_ticks, dtype=int)  
plt.xticks(df_selected.index[indices], rotation=45) 

# Affichage 
plt.legend(loc="upper left", bbox_to_anchor=(1, 1), ncol=2)  # Légende en dehors du graphique
plt.grid(True)
plt.gca().yaxis.set_major_formatter(ticker.StrMethodFormatter("{x:,.0f}")) #permet de ne pas avoir d'écriture scientifique
plt.show()


### Données d'heures travaillées <a class="anchor" id="partie32"></a>

Voici l'évolution des heures travaillées en France.

In [ ]:
plt.figure(figsize=(14, 6))  # Taille du graphique

plt.plot(df.index, df['WH_France'], marker='o', linestyle='-')

plt.xlabel("Période")
plt.ylabel("Heures travaillées en France")
plt.title("Évolution des heures travaillées en France (1995-2024)")

# Sélectionner un sous-ensemble des labels de l'axe X pour éviter la surcharge
num_ticks = 10 
indices = np.linspace(0, len(df.index) - 1, num_ticks, dtype=int)
plt.xticks(df.index[indices], rotation=45)
plt.grid(True)
plt.show()